In [1]:
require 'nn';
require 'Attention';
require 'Association';
require 'image';
require 'gnuplot';

# Data Preparation 

In [2]:
--Three Categories: 
-- 1=Big/0=Small; 1=Rectangle/0=Triangle; 1=Black/0=White
data = torch.Tensor(8,3)
data[1] = torch.Tensor({1,1,1})
data[2] = torch.Tensor({0,1,1})
data[3] = torch.Tensor({1,0,1})
data[4] = torch.Tensor({1,1,0})
data[5] = torch.Tensor({0,0,1})
data[6] = torch.Tensor({1,0,0})
data[7] = torch.Tensor({0,1,0})
data[8] = torch.Tensor({0,0,0})

alcove = nn.Sequential()
attention = nn.Attention(3, 8, data, 6.5, 1,1)
association = nn.Association(2,8)

alcove:add(attention)
alcove:add(association)


# Training

In [3]:
probabilities = torch.Tensor(50)
target = torch.Tensor(2)
k = 1
for i = 1,50 do
    for j = 1,8 do
        a_out = alcove:forward(data[(j+i)%8 + 1])
        if data[(j+i)%8 +1][1] == 1  then
            target[1] = math.max(1,a_out[1])
            target[2] = math.min(-1,a_out[2])
            if j == 2 then
                probabilities[k] = (math.exp(a_out[2]*2.0))/torch.sum(torch.mul(a_out,2.0):exp())
                k = k+1
            end    
        else
            target[1] = math.min(-1,a_out[1])
            target[2] = math.max(1,a_out[2])
            if j == 2 then
                probabilities[k] = (math.exp(a_out[1]*2.0))/torch.sum(torch.mul(a_out,2.0):exp())
                k = k+1
            end
        end
        alcove:backward(data[(j+i)%8 + 1],target-a_out)
    end
    attention:updateParameters(-0.0033)
    association:updateParameters(0.03)
    alcove:zeroGradParameters()
end
gnuplot.plot(probabilities)